<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.856 · Análisis de sentimientos y redes sociales</p>
<p style="margin: 0; text-align:right;">Máster universitario de Ciencia de datos (<i>Data Science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# Análisis de sentimientos y redes sociales
## PLA4: Representación de redes sociales como grafos

## Adquisición de datos de redes sociales

Los procesos de adquisición de datos de redes sociales son muy diversos. En esta unidad, veremos ejemplos de adquisición de datos de redes sociales siguiendo tres enfoques distintos:

- la descarga **directa** de los datos,
- la obtención de datos mediante peticiones a **API**s de terceros,
- la recolección de datos a partir de **web crawling**.

Con relación a la primera alternativa, la descarga directa de los datos, presentaremos algunos de los repositorios de datos de redes sociales más populares.

En cuanto a la interacción con APIs de terceros, veremos como usar librerías Python para interactuar con APIs.

Respecto al *web crawling*, veremos cómo utilizar la librería [Scrapy](https://scrapy.org/) para construir un pequeño *web crawler* que capture datos de nuestro interés.

## 1. Descarga directa de los datos

En algunas ocasiones tendremos la oportunidad de trabajar con conjuntos de datos que ya han sido recopilados anteriormente y que se encuentran directamente disponibles para su descarga. Será el caso, por ejemplo, de datos recopilados por algún otro analista, datos cedidos directamente por el proveedor de la red social, conjuntos de datos públicos utilizados como referencia, etc.

Algunos de los sitios de referencia para descargar datos de redes sociales son:
- El apartado dedicado a [redes sociales del Network Repository](http://networkrepository.com/soc.php) contiene múltiples conjuntos de datos de estas redes y ofrece, además, una herramienta para visualizar de manera interactiva las redes y estadísticas sobre ellas.
- [Stanford Large Network Dataset Collection](http://snap.stanford.edu/data/), una recopilación de conjuntos de datos en forma de red creada por Jure Leskovec y Andrej Krevl. Contiene datos de redes sociales como Facebook, Twitter o Google+, además de otros tipos de datos que tienen estructura de red, por ejemplo, redes de colaboración entre científicos o redes de ordenadores.
- [Mark Newman Network Datasets](http://www-personal.umich.edu/~mejn/netdata/), es también un listado de conjuntos de datos en forma de red, en este caso recopilados por Mark Newman. Contiene algunos conjuntos de datos de redes sociales, aunque la mayoría pertenecen a otros tipos de redes.
- El repositorio [KONECT de la Universidad de Koblenz](http://konect.uni-koblenz.de/networks/) pone a disposición pública datos de redes, algunos de los cuáles pertenecen a redes sociales (los encontraremos dentro de la categoría *social*). 

En estos casos, la descarga de los datos se realiza directamente (usando, por ejemplo, un navegador web, un cliente [ftp](https://en.wikipedia.org/wiki/File_Transfer_Protocol), un cliente de [BitTorrent](https://en.wikipedia.org/wiki/BitTorrent) o a través del propio Python), y simplemente será necesario leer el fichero descargado para poder empezar a trabajar con los datos. Veamos un ejemplo muy sencillo de la descarga directa de un fichero comprimido con datos de la red Google+.


In [1]:
import requests, zipfile, sys
zip_file_url = "http://nrvis.com/download/data/soc/soc-gplus.zip"

if sys.version_info[0] == 2:
    import StringIO
    # Descargamos un fichero comprimido con los datos del Network Repository
    r = requests.get(zip_file_url, stream=True)

    # Extraemos el contenido del zip en la carpeta data
    z = zipfile.ZipFile(StringIO.StringIO(r.content))
elif sys.version_info[0] == 3:
    # python3
    import io
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
else:
    print("Unrecognized Python version")
    exit()
    
z.extractall("./data")

In [2]:
# Abrimos el fichero descomprimido y mostramos las 10 primeras líneas
with open("./data/soc-gplus.edges", 'r') as f:
    for _ in range(10):
        read_data = f.readline()
        print(read_data)


% asym unweighted

% 39242 131 23591

1 2 

1 3 

1 4 

1 5 

1 6 

1 7 

1 8 

1 9 



## 2. API de terceros

Muchas redes sociales ponen a disposición de los desarrolladores una API (del inglés, *Application Programming Interface*) que permite interactuar con la red, ya sea para descargar contenidos existentes o para publicar nuevos contenidos. En esta sección, veremos cómo podemos descargar contenido de redes sociales a través de sus API.

Una API es un conjunto de métodos de comunicación entre varios componentes de software. Las API facilitan el trabajo de integración de programas ya que permiten ofrecer una interfaz clara y bien especificada con la que interactuar con una aplicación, ocultando los detalles de la implementación concreta y exponiendo únicamente funciones específicas de interés.

La definición de API es muy genérica y podemos encontrar API en muchos contextos. En esta unidad, nos centraremos en el uso de API web para la adquisición de datos de servicios de terceros. Las API web se definen habitualmente como un conjunto de peticiones HTTP junto con la especificación de la estructura de los datos de las respuestas, normalmente en formato JSON o XML.

El uso de API web está muy extendido actualmente para interactuar con grandes proveedores de servicios en Internet. Algunos ejemplos de API populares son las de Google maps, YouTube, Spotify, Twitter o Facebook.

Para interactuar con una web API realizaremos una **petición HTTP**. A su vez, el servidor nos responderá con un mensaje de respuesta HTTP. Las peticiones y respuestas HTTP se estructuran en tres partes:

* Una línea inicial de petición, que incluye la acción que hay que realizar (el método de la petición) y la URL del recurso, en las peticiones; y el código de estado y el mensaje asociado, en el caso de las respuestas.
* La cabecera, que incluye metadatos con varias finalidades, por ejemplo, para describir el contenido, realizar la autenticación o controlar las cookies.
* Una línea en blanco que separa la cabecera del cuerpo.
* El cuerpo, que puede estar vacío o contener datos.

Las acciones o métodos más usados en interacción con web API son:

* `GET`: permite obtener información del recurso especificado.
* `POST`: permite enviar datos al recurso especificado.
* `PUT`: carga datos actualizando los ya existentes en el recurso especificado.
* `DELETE`: elimina información del recurso especificado.

Dos de los formatos más habituales para incluir datos en las respuestas de las web API son **JSON** y **XML**. Ambos formatos tienen varias propiedades en común. En primer lugar, fueron diseñados para ser legibles tanto por humanos como por ordenadores, lo que los hace ideales en este contexto. En segundo lugar, ambos incorporan información sobre la estructura de los datos que codifican. Finalmente, ambos almacenan los datos en texto claro. Sin embargo, ambos lenguajes presentan múltiples diferencias.

El formato **XML** (del inglés, eXtensible Markup Language) es un lenguaje de marcas que utiliza un conjunto de etiquetas no predefinido. Los documentos XML tienen un único elemento raíz del cual pueden colgar otros elementos. Los elementos se delimitan con una etiqueta inicial y una etiqueta final.

El formato **JSON** (del inglés, JavaScript Object Notation) es un subconjunto de la notación de objetos javascript. JSON se basa en dos estructuras de datos, el array y el objeto, que serían equivalentes a las listas y los diccionarios de Python que ya hemos introducido. 

### 2.1 La API de Twitter

**Twitter** es una red social creada a principios de 2006 que proporciona una plataforma para intercambiar pequeños mensajes de texto, conocidos como tweets. Tradicionalmente los tweets estaban limitados a 140 caracteres, pero en la actualidad están permitidos hasta 280.

Twitter ofrece varias [API](https://developer.twitter.com/en/docs/api-reference-index) que permiten obtener datos de la red. En esta unidad, trabajaremos con una de ellas, la API REST.

Los datos devueltos por las API de Twitter se encuentran codificados usando JavaScript Object Notation (JSON). De todos modos, usaremos una librería de Python para interactuar con la API, Tweepy, que ya se encarga de procesar el JSON de respuesta de Twitter y crea objetos de Python con el resultado de cada consulta, por lo que no tendremos que tratar con el formato de los datos manualmente.

### 2.1.1 Autenticación con la API de Twitter

Twitter requiere autenticación para poder utilizar su API. Por este motivo, el primer paso que se ha de realizar para poder obtener datos de Twitter a través de su API es conseguir unas credenciales adecuadas. En esta sección, describiremos cómo obtener credenciales para acceder a la API de Twitter.

Para empezar, es necesario disponer de una cuenta en Twitter. Para poder ejecutar los ejemplos del notebook, necesitaréis por lo tanto tener una cuenta de Twitter. Podéis utilizar vuestra cuenta personal, si ya disponéis de ella, para solicitar los permisos de desarrollador que nos permitirán interactuar con la API. En caso contrario (o si preferís no usar vuestra cuenta personal), podéis crearos una cuenta de Twitter nueva. El proceso es muy sencillo:
1. Acceder a [Twitter](http://www.twitter.com).
2. Pulsar sobre *Sign up for Twitter* y seguir las indicaciones para completar el registro.

Después, habrá que solicitar convertir la cuenta recién creada (o vuestra cuenta personal) en una cuenta de desarrollador. Para hacerlo, hay que seguir los siguientes pasos:
1. Acceder al [panel de desarolladores de Twitter](https://developer.twitter.com/).
2. Clicar sobre *Apply*.
3. Clicar sobre *Apply for a developer account*.
3. Pulsar *Continue*.
4. Indicar porqué queréis disponer de una cuenta de desarrollador (podéis explicar que estáis siguiendo un curso de análisis de redes sociales y que vamos a realizar unas pruebas con la API de Twitter).

Para poder realizar este proceso satisfactoriamente, necesitaréis que vuestra cuenta disponga de un número de teléfono asociado verificado. En caso contrario, veréis que os aparecerá un mensaje para que verifiquéis vuestro teléfono. 

Finalmente, una vez ya disponemos de una cuenta en Twitter, será necesario registrar una nueva aplicación. Para hacerlo, es necesario seguir los siguientes pasos:
1. Acceder al [panel de desarolladores de Twitter](https://developer.twitter.com/en/apps).
2. Pulsar sobre *Create new app*.
3. Rellenar el formulario con los detalles de la aplicación. En concreto, necesitaréis proporcionar como mínimo los campos:
    * *App name*
    * *Application description*
    * *Website URL*
    * *Tell us how this app will be used*

El campo *Website* debe contener una URL válida (por ejemplo, el enlace a vuestro perfil de Twitter).

Una vez creada la aplicación, podéis acceder a la pestaña *Keys and access tokens*. Allí se encuentran las credenciales recién creadas para vuestra aplicación, que utilizaremos para autenticarnos y poder utilizar la API de Twitter. Veréis que ya tenéis las claves *Consumer API keys* disponibles. Además, será necesario pulsar sobre *Create* en la sección *Access token & access token secret* para obtener también ambos *tokens*. Los cuatro valores serán usados para autenticar nuestra aplicación:
* API / Consumer Key
* API / Consumer Secret
* Access Token
* Access Token Secret

### 2.1.2 Tweepy

Una de las librerías más populares para interactuar con la API de Twitter es [Tweepy](http://www.tweepy.org/). La librería permite interactuar con la API de Twitter de una manera sencilla, ya que encapsula los métodos HTTP de la API en métodos de Python, que pueden ser llamados directamente. Encontraréis la documentación de la librería en el siguiente [enlace](http://tweepy.readthedocs.io).

Para simplificar la interacción con la librería, creamos una función `get_twitter_api`, que crea un objeto de la [clase api de tweepy](http://tweepy.readthedocs.io/en/v3.5.0/api.html#tweepy-api-twitter-api-wrapper) inicializándolo con nuestras credenciales o bien devuelve el objeto creado anteriormente (a no ser que se indique lo contrario con el parámetro `reset`).

In [3]:
# Importamos la librería Tweepy
import tweepy

# Definimos la variable API, que almacenará el objeto tweepy.API inicializado con nuestras credenciales
api = None

# Definimos la función get_twitter_api, que devolverá el objeto tweepy.API. La función creará
# un objeto tweepy.API si este no se ha creado previamente (o si lo indicamos explícitamente 
# con reset=True) o bien devolverá el objeto creado anteriormente.
def get_twitter_api(reset=False, consumer_key=None, consumer_secret=None, access_token=None, access_secret=None):
    global api
    if not api or reset:
        print("Initializing tweepy API...")
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        api = tweepy.API(auth)
    return api

A continuación, creamos el objeto API que utilizaremos para interactuar con Twitter, especificando las credenciales de la aplicación que hemos creado anteriormente.

In [4]:
# Creamos un objeto tweepy.API con la función get_twitter_api
# IMPORTANTE: es necesario incluir las credenciales de acceso que hayáis obtenido al crear vuestra App
# para ejecutar el resto de ejemplos del notebook.
api = get_twitter_api(consumer_key = 'tuzlLXJ7Cjcw9ysoeEhT60ENh', 
                      consumer_secret = 'cO9hqRaP94IP66ssVqHHqVyibrLMQ1et8XHbhczy2eBW6rFFiI',
                      access_token = '1029309220234186752-GikHFK2syTi2kiH0bVx8fMNhdP8IDJ',
                      access_secret = 'AJIuznR3PXwPh30GZ8YYqyQfpTyjkZrnu5bglzGUSaN5F'
                      )
api

Initializing tweepy API...


### 2.1.3 Los usuarios de Twitter

Uno de los objetos básicos de la API de Twitter es el que representa a los **usuarios** de la red. Todos los usuarios de Twitter tienen un identificador único, que es un valor numérico que Twitter utiliza para hacer referencia al usuario dentro de la red. Aunque este identificador resulta muy práctico para seleccionar a un usuario concreto de forma inequívoca, es difı́cil de recordar para las personas, por lo que Twitter permite definir tanto un nombre de usuario, que se utilizará para iniciar sesión en la red, como un nombre de perfil, que se mostrará en el perfil y que podrá cambiarse con el tiempo, si el usuario ası́ lo desea.

Además de información de identificación, el objeto usuario de Twitter dispone de muchos otros atributos, por ejemplo, la url y la descripción que el usuario incluye en su perfil, el número de seguidores y de amigos, el número de tweets que ha enviado, etc. En la documentación de la API de Twitter, encontraréis una descripción de todos los atributos que tienen los objetos de [usuario](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object) de la API de Twitter. 

Veamos cómo podemos usar el método [`get_user`](http://docs.tweepy.org/en/v3.5.0/api.html?highlight=get_user#API.get_user) de Tweepy para obtener datos de un usuario de Twitter:

In [5]:
# Recuperamos el objeto api
api = get_twitter_api()

# Obtenemos el objeto user, que representa a un usuario de Twitter.
# Se puede acceder a un usuario por id de usuario o nombre de usuario (screen_name)
user = api.get_user('twitter')

print("El tipo de datos de la variable user es: {}".format(type(user)))
print("El nombre de usuario es: {}".format(user.screen_name))
print("El id de usuario es: {}".format(user.id))

El tipo de datos de la variable user es: <class 'tweepy.models.User'>
El nombre de usuario es: Twitter
El id de usuario es: 783214


In [6]:
# Mostramos algunos atributos del usuario recuperado
print("El número de seguidores es: {}".format(user.followers_count))
print("El número de amigos es: {}".format(user.friends_count))
print("El número de tweets es: {}".format(user.statuses_count))

El número de seguidores es: 56606985
El número de amigos es: 35
El número de tweets es: 11543


In [7]:
# Visualizamos los atributos de la instancia user
print(user)

User(_api=<tweepy.api.API object at 0x105d5b1d0>, _json={'id': 783214, 'id_str': '783214', 'name': 'Twitter', 'screen_name': 'Twitter', 'location': 'Everywhere', 'profile_location': None, 'description': 'What’s happening?!', 'url': 'https://t.co/TAXQpsHa5X', 'entities': {'url': {'urls': [{'url': 'https://t.co/TAXQpsHa5X', 'expanded_url': 'https://about.twitter.com/', 'display_url': 'about.twitter.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 56606985, 'friends_count': 35, 'listed_count': 90738, 'created_at': 'Tue Feb 20 14:35:54 +0000 2007', 'favourites_count': 6210, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 11543, 'lang': None, 'status': {'created_at': 'Fri Aug 30 18:18:14 +0000 2019', 'id': 1167501799324082178, 'id_str': '1167501799324082178', 'text': '@WittLowry #TEAMWITT is on fire', 'truncated': False, 'entities': {'hashtags': [{'text': 'TEAMWITT', 'indices': [11, 20]}], 'symb

Las redes sociales se caracterizan por permitir que sus usuarios creen relaciones explı́citas dentro de la propia red. Twitter implementa estas relaciones explícitas mediante las relaciones de seguimiento entre usuarios. Así pues, los usuarios de Twitter pueden seguir a otros usuarios de la red.

Las relaciones entre usuarios en Twitter no son bidireccionales, es decir, Twitter diferencia entre la relación en la que Alice sigue a Bob y la relación en la que Bob sigue a Alice. Cuando el usuario Alice sigue a Bob, decimos que Alice es una seguidora o *follower* de Bob. A la vez, Bob es entonces un amigo o *friend* de Alice.

Podemos recuperar a los seguidores (*followers*) de un usuario y a los usuarios a los que este sigue (amigos o *friends*) usando la API de Twitter: 

In [8]:
# Obtenemos los identificadores de los followers y los friends del usuario de Twitter
followers_ids = api.followers_ids("twitter")
friends_ids = api.friends_ids("twitter")
print("El id del primer follower es: {}".format(followers_ids[0]))
print("El id del primer friend es: {}".format(friends_ids[0]))

# También podemos recuperar los followers o friends como objetos User de Tweepy
followers = api.followers("twitter")
friends = api.friends("twitter")
print("El tipo de datos de un follower es: {}".format(type(followers[0])))
print("El tipo de datos de un friend es: {}".format(type(friends[0])))

# Así, podemos acceder directamente a los atributos de los followers y friends del usuario de Twitter
a_follower = followers[0]
print("Datos del primer follower del usuario de Twitter:")
print("\tEl nombre de usuario es: {}".format(a_follower.screen_name))
print("\tEl id de usuario es: {}".format(a_follower.id))
print("\tEl número de seguidores es: {}".format(a_follower.followers_count))
print("\tEl número de amigos es: {}".format(a_follower.friends_count))
print("\tEl número de tweets es: {}".format(a_follower.statuses_count))


El id del primer follower es: 1150102360250707968
El id del primer friend es: 2408170710
El tipo de datos de un follower es: <class 'tweepy.models.User'>
El tipo de datos de un friend es: <class 'tweepy.models.User'>
Datos del primer follower del usuario de Twitter:
	El nombre de usuario es: staywith_mie
	El id de usuario es: 1150102360250707968
	El número de seguidores es: 0
	El número de amigos es: 10
	El número de tweets es: 0


Dada una relación entre dos usuarios, también podemos obtener datos adicionales de esta usando el método [`showfriendship`](http://docs.tweepy.org/en/v3.5.0/api.html?highlight=show_friendship#API.show_friendship):


In [9]:
api.show_friendship(source_id=1006576427423227905, target_id=783214)

(Friendship(_api=<tweepy.api.API object at 0x105d5b1d0>, _json={'id': 1006576427423227905, 'id_str': '1006576427423227905', 'screen_name': 'murcianomario', 'following': False, 'followed_by': False, 'live_following': False, 'following_received': None, 'following_requested': None, 'notifications_enabled': None, 'can_dm': False, 'blocking': None, 'blocked_by': None, 'muting': None, 'want_retweets': None, 'all_replies': None, 'marked_spam': None}, id=1006576427423227905, id_str='1006576427423227905', screen_name='murcianomario', following=False, followed_by=False, live_following=False, following_received=None, following_requested=None, notifications_enabled=None, can_dm=False, blocking=None, blocked_by=None, muting=None, want_retweets=None, all_replies=None, marked_spam=None),
 Friendship(_api=<tweepy.api.API object at 0x105d5b1d0>, _json={'id': 783214, 'id_str': '783214', 'screen_name': 'Twitter', 'following': False, 'followed_by': False, 'following_received': None, 'following_requested':

### 2.1.4 Los tweets

Los **tweets** son las unidades básicas de comunicación entre los usuarios de Twitter. Todos los tweets tienen asociado un identificador único dentro de la red, y contienen el texto de este. 

Además del identificador y del texto que comunican, también contienen información sobre el momento en el que fueron creados (el día y la hora), así como datos de la aplicación utilizada por el usuario para crear el tweet (los tweets creados a través de la interfaz web de Twitter contienen simplemente la cadena "web" como identificador de aplicación de origen), entre otros atributos. En la documentación de la API de Twitter, encontraréis una descripción de todos los atributos que tienen los objetos [tweet](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object) de la API de Twitter. 


Veamos cómo podemos usar el método [`get_status`](http://docs.tweepy.org/en/v3.5.0/api.html?highlight=get_status#API.get_status) de Tweepy para obtener datos de un usuario de Twitter:

In [10]:
# Obtenemos el objeto status, que representa un tweet.
# Se puede acceder a un tweet a través de su id
status = api.get_status(990518384407318528)

print("El tipo de datos de la variable status es: {}".format(type(status)))
print("El id del tweet es: {}".format(status.id))
print(u"El texto del tweet es: {}".format(status.text))


El tipo de datos de la variable status es: <class 'tweepy.models.Status'>
El id del tweet es: 990518384407318528
El texto del tweet es: Las Claves del RGPD en nueve Puntos! Con la entrada en vigor del RGPD las empresas deben tomarse muy en serio la pr… https://t.co/Pe4QzyUJA3


Notad que el texto del tweet está codificado usando [unicode](https://docs.python.org/2/howto/unicode.html):

In [11]:
if sys.version_info[0] == 2:
    isinstance(status.text, unicode)
elif sys.version_info[0] == 3:
    # in Python 3, strings are unicode:
    isinstance(status.text, str)

Como hemos visto, los usuarios de Twitter crean relaciones explícitas entre ellos mediante las relaciones de seguimiento. De un modo similar, los tweets pueden también crear relaciones explícitas entre ellos. Este es el caso de los *retweets*, que suponen la republicación de un tweet, los *quotes*, que permiten también republicar un tweet pero añadiendo un comentario, y los *replies*, que permiten responder a un tweet.

In [12]:
# Recuperamos un tweet que republica otro tweet, es decir, un retweet
a_retweet = api.get_status(1030046532064747522)

# Mostramos datos del tweet que se está republicando
retweeted_status = a_retweet.retweeted_status
print("El tweet está republicando el tweet con id : {}".format(retweeted_status.id))
print("El tweet está republicando un tweet del usuario con id : {}".format(retweeted_status.user.id))
print("El tweet original tiene como texto: {}".format(retweeted_status.text))

El tweet está republicando el tweet con id : 1029823889365450752
El tweet está republicando un tweet del usuario con id : 1469101279
El tweet original tiene como texto: Bitcoin Q&amp;A: Why I'm against ETFs https://t.co/8mQtKJgy0L


In [13]:
# Recuperamos un tweet que responde a otro tweet, es decir, un reply
a_reply = api.get_status(1029918429992689665)

# Mostramos a qué tweet se responde y quien fue su autor
print("El tweet está repondiendo al tweet con id : {}".format(a_reply.in_reply_to_status_id))
print("El tweet está repondiendo al usuario con id : {}".format(a_reply.in_reply_to_user_id))

El tweet está repondiendo al tweet con id : 1029894625249509376
El tweet está repondiendo al usuario con id : 2568108282


In [14]:
# Recuperamos un tweet que comenta otro tweet, es decir, un quote
a_quote = api.get_status(1029401266999447552)

# Mostramos el tweet que se comenta
print("El tweet está comentando el tweet con id : {}".format(a_quote.quoted_status_id))
quoted_status = a_quote.quoted_status
print("El tweet está comentando al usuario con id : {}".format(quoted_status.user.id))
print(u"El tweet original tiene como texto: {}".format(quoted_status.text))

El tweet está comentando el tweet con id : 1029112398555176961
El tweet está comentando al usuario con id : 297564170
El tweet original tiene como texto: WOW! @CapitalOne Just froze my bank account (and all $309,000 of cash I had collected for an upcoming house purchas… https://t.co/khmv9vfZvp


Notad que los tipos de datos que se obtienen en cada caso no son siempre los mismos, por lo que habrá que ir con cuidado al trabajar con estos métodos. Por ejemplo, `retweeted_status` es un objeto tweet de Tweepy, mientras que `quoted_status` es un diccionario.

In [15]:
print("El tipo de datos de retweeted_status es: {}".format(type(retweeted_status)))
print("El tipo de datos de quoted_status es: {}".format(type(quoted_status)))

El tipo de datos de retweeted_status es: <class 'tweepy.models.Status'>
El tipo de datos de quoted_status es: <class 'tweepy.models.Status'>


Además, Tweepy tampoco es consistente al tratar el acceso a campos relativos a características no presentes en el tweet, un detalle que también será necesario tener en cuenta a la hora de incorporar estas funcionalidades a nuestros programas:

In [16]:
# Accedemos a in_reply_to_status_id de un retweet:
print(a_retweet.in_reply_to_status_id)

# Accedemos a retweeted_status de un reply:
try:
    a_reply.retweeted_status
except AttributeError:
    print("AttributeError")
    
# Disponemos de un booleano que nos indica si un tweet es un quote
# pero este booleano no existe en caso de replies y retweets
print("El tweet a_quote es un comentario: {}".format(a_quote.is_quote_status))
print("El tweet a_reply es un comentario: {}".format(a_reply.is_quote_status))

None
AttributeError
El tweet a_quote es un comentario: True
El tweet a_reply es un comentario: False


Además, los tweets obtenidos de la API de Twitter contienen también [entidades](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/entities-object), un conjunto de metadatos adicionales que describen datos que se incluyen habitualmente en tweets como etiquetas (*hashtags*), enlaces, menciones a otros usuarios, ficheros incrustados, etc. Podríamos obtener estos datos parseando manualmente el texto del tweet, pero la API nos los devuelve también ya procesados.


In [17]:
# Mostramos las entidades del tweet status
status.entities

{'hashtags': [],
 'symbols': [],
 'user_mentions': [],
 'urls': [{'url': 'https://t.co/Pe4QzyUJA3',
   'expanded_url': 'https://twitter.com/i/web/status/990518384407318528',
   'display_url': 'twitter.com/i/web/status/9…',
   'indices': [117, 140]}]}

Algunas veces, los tweets también contienen información sobre el [lugar](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/geo-objects) desde el que fueron enviados a la red, ya sea en forma de puntos exactos, indicados a partir de sus coordenadas geográficas, o bien a través de "lugares" delimitados sobre una zona más amplia.

In [18]:
# Ejemplo de un tweet vinculado a un lugar de Twitter
status_with_place = api.get_status(1030787428125032448)
print(status_with_place.place)

Place(_api=<tweepy.api.API object at 0x105d5b1d0>, id='006523c50dfe9086', url='https://api.twitter.com/1.1/geo/id/006523c50dfe9086.json', place_type='city', name='Quezon City', full_name='Quezon City, National Capital Region', country_code='PH', country='Republic of the Philippines', contained_within=[], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x105d5b1d0>, type='Polygon', coordinates=[[[120.989705, 14.5893763], [121.1357656, 14.5893763], [121.1357656, 14.7766484], [120.989705, 14.7766484]]]), attributes={})


In [19]:
# Ejemplo de un tweet con coordenadas geográficas
status_with_coord = api.get_status(1030808412286636034)
print(status_with_coord.coordinates)

{'type': 'Point', 'coordinates': [-74.0064, 40.7142]}


## 3. Web crawling

A veces necesitaremos acceder a datos que están publicados en internet pero para los cuáles no hay un fichero que podamos descargar ni una API a la que podamos hacer consultas. En estos casos, puede ser útil crear un pequeño *crawler*, es decir, un programa que rastree las páginas web que contengan los datos de interés, los parsee y los recopile en algún fichero de almacenamiento persistente, para que puedan ser usados posteriormente para nuestros análisis.

<span style="color:blue">[Scrapy](httpscrapy.org/) es una librería de Python que provee de un _framework_ para la extracción de datos de páginas web. Podéis ver una introducción al uso de Scrapy así como un ejemplo de uso en el notebook de la Unidad 5 de la asignatura Programación en Python.</span>

Veamos un ejemplo de cómo usar Scrapy para extraer datos de redes sociales extrayendo los contactos de un usuario de Flickr, una red centrada en la compartición de imágenes y vídeos.

Podemos obtener un listado de los contactos de un usuario de Flickr accediendo a su perfil. Así, por ejemplo, encontraremos los contactos del usuario `egofreed` en la siguiente url:

https://www.flickr.com/people/egofreed/contacts

Si queremos extraer los nombres de los usuarios que conforman los contactos de este usuario, deberemos observar el html de esta página, para ver dónde podemos encontrar esta información. 

Como puede apreciarse, los datos que queremos recopilar (los nombres de usuario de los contactos) se encuentran en múltiples atributos. Si nos fijamos en el primer contacto:

podemos ver que el nombre de usuario, `cristinamerz`, aparece en tres enlaces distintos. Por lo tanto, podemos seleccionar cualquiera de ellos para extraer el dato de interés. En nuestro caso, seleccionaremos el último de los enlaces. Así pues, crearemos una expresión xpath que seleccione el atributo href de un hipervínculo (un elemento señalado con la etiqueta `<a>`) que tiene como atributo `rel` el valor «contact» y, de las tres coincidencias que obtendremos, nos quedaremos únicamente con la última. La expresión xpath que nos permite hacer esta selección es:

```
//a[@rel="contact"][2]/@href
```

Una vez tenemos la expresión xpath que selecciona los datos de interés, ya podemos programar nuestra araña para que los extraiga.

**Nota**: Si la ejecución del *crawler* os devuelve un error `ReactorNotRestartable`, reiniciad el kernel del notebook (en el menú, `Kernel` - `Restart`).


In [20]:
# Importamos scrapy.
import scrapy
from scrapy.crawler import CrawlerProcess

# Creamos la araña.
class uoc_flickr_spider(scrapy.Spider):
    
    # Asignamos un nombre a la araña.
    name = "uoc_flickr_spider"
    
    # Indicamos la url que queremos analizar en primer lugar.
    start_urls = [
        "https://www.flickr.com/people/egofreed/contacts"
    ]

    # Definimos el analizador.
    def parse(self, response):
        # Extraemos el nombre de usuario del contacto.
        for contact in response.xpath('//a[@rel="contact"][2]/@href'):
            yield {
                # Nos quedamos únicamente con el nombre de usuario
                'contact_username': contact.extract()[8:-1]
            }
            
if __name__ == "__main__":

    # Creamos un crawler.
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'DOWNLOAD_HANDLERS': {'s3': None},
        'LOG_ENABLED': True
    })

    # Inicializamos el crawler con nuestra araña.
    process.crawl(uoc_flickr_spider)
    
    # Lanzamos la araña.
    process.start()

2019-09-02 17:48:08 [scrapy.utils.log] INFO: Scrapy 1.7.2 started (bot: scrapybot)
2019-09-02 17:48:08 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.1, Python 3.6.7 (v3.6.7:6ec5cf24b7, Oct 20 2018, 03:02:14) - [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.7, Platform Darwin-18.7.0-x86_64-i386-64bit
2019-09-02 17:48:08 [scrapy.crawler] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-09-02 17:48:08 [scrapy.extensions.telnet] INFO: Telnet Password: 140a4f85580b3f7a
2019-09-02 17:48:08 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-09-02 17:48:08 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloader

En el log de la ejecución de Scrapy podremos ver el resultado de nuestro *crawling*, con los nombres de usuario de los contactos de Flickr del usuario `egofreed`.